# 02 Transfer core loop
Hypothesis: source initialization improves sample efficiency and convergence speed.

### Expected Outcome
Transfer methods should beat scratch in early and final target accuracy.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
methods = ['scratch', 'feature_extraction', 'gradual_unfreeze', 'naive_finetune']
LAB.run('transfer_core_related.yaml', '02_core_loop', fast_dev_run=FAST_DEV_RUN)
frames = LAB.load_methods(methods)
summary = pd.DataFrame([
    {
        'method': m,
        'best_target_acc': float(df['target_test_acc'].max()),
        'final_target_acc': float(df['target_test_acc'].iloc[-1]),
        'final_feature_drift': float(df['feature_drift'].iloc[-1]),
    }
    for m, df in frames.items()
]).sort_values('best_target_acc', ascending=False)
summary

In [ ]:
fig, ax = plt.subplots(figsize=(6.6, 3.7))
for m, df in frames.items():
    ax.plot(df['epoch'], df['target_test_acc'], marker='o', label=m)
ax.set_title('Target accuracy by method')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '02_target_acc_by_method.png')

fig, ax = plt.subplots(figsize=(6.6, 3.7))
for m, df in frames.items():
    ax.plot(df['epoch'], df['feature_drift'], marker='o', label=m)
ax.set_title('Feature drift by method')
ax.set_xlabel('epoch')
ax.set_ylabel('feature_drift')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '02_feature_drift_by_method.png')

### Loop Mapping
The only variable is adaptation policy after source pretraining.